# Use RAY with TensorFlow Workers <font color='grey'> (*Self-Contained*) </font>

In [1]:
import datetime,gym,os,pybullet_envs,time,ray
import numpy as np
np.set_printoptions(precision=2)
import tensorflow as tf
from util import suppress_tf_warning
suppress_tf_warning()
print ("Packaged loaded. TF version is [%s]."%(tf.__version__))



Packaged loaded. TF version is [1.14.0].


### Initialize Ray

In [2]:
n_cpus = 5
ray.init(num_cpus=n_cpus)
print ("RAY initialized with [%d] cpus."%(n_cpus))

2020-06-15 09:58:14,235	WARNING services.py:586 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-06-15 09:58:14,238	INFO resource_spec.py:212 -- Starting Ray with 14.89 GiB memory available for workers and up to 7.46 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-06-15 09:58:14,612	INFO services.py:1078 -- View the Ray dashboard at localhost:8265


RAY initialized with [5] cpus.


### TF Model Creator. Note that `import tensorflor as tf` should be inside the function.

In [3]:
def create_model(x_dim,y_dim):
    # import tensorflow as tf
    import tensorflow as tf
    
    # Build a simple two-layer model
    x_ph = tf.placeholder(tf.float32,shape=[None,x_dim])
    with tf.variable_scope('main'):
        net = tf.layers.dense(inputs=x_ph,units=32,activation=tf.nn.relu)
        y = tf.layers.dense(inputs=net,units=y_dim,activation=None)
    def get_vars(scope):
        return [x for x in tf.compat.v1.global_variables() if scope in x.name]
    g_vars = get_vars('main')
    
    # Have own session
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    
    # Initialize weights
    sess.run(tf.global_variables_initializer())
    return x_ph,y,g_vars,sess

### Rollout Worker with a TF Model inside

In [4]:
@ray.remote
class RolloutWorkerClass(object):
    def __init__(self,worker_id=0,x_dim=5,y_dim=2):
        self.worker_id = worker_id
        # Make TF Model
        self.x_ph,self.y,self.g_vars,self.sess = create_model(x_dim=x_dim,y_dim=y_dim)
    def get_weights(self):
        """
        Get weights of 'g_vars'
        """
        return self.sess.run(self.g_vars)
    def set_weights(self,weight_list):
        """
        Set weights of 'g_vars'
        """
        for g_idx,g_var in enumerate(self.g_vars):
            self.sess.run(tf.assign(g_var,weight_list[g_idx]))
    def rollout(self,x):
        return self.sess.run(self.y,feed_dict={self.x_ph:x})

### Initialized Workers

In [5]:
x_dim,y_dim = 64,8
n_workers = 5
workers = [RolloutWorkerClass.remote(worker_id=i,x_dim=x_dim,y_dim=y_dim) for i in range(n_workers)]
print ("[%d] workers initialized."%(n_workers))

[5] workers initialized.


### Initialize an external TF model whose weights will be shared among workers

In [6]:
_,_,g_vars,sess = create_model(x_dim=x_dim,y_dim=y_dim)
weights = sess.run(g_vars)

### Rollouts and Check the results

In [7]:
x_rand = np.random.rand(1,x_dim)

In [8]:
rollout_list = [worker.rollout.remote(x=x_rand) for worker in workers] # non-block
rollout_res_list = ray.get(rollout_list)

(pid=8793) WARNING:tensorflow:From <ipython-input-3-00d26d89fd58>:8: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
(pid=8793) Instructions for updating:
(pid=8793) Use keras.layers.dense instead.
(pid=8793) WARNING:tensorflow:From /home/sj/.adt/venv-adt/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
(pid=8793) Instructions for updating:
(pid=8793) Call initializer instance with the dtype argument instead of passing it to the constructor
(pid=8789) WARNING:tensorflow:From <ipython-input-3-00d26d89fd58>:8: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
(pid=8789) Instructions for updating:
(pid=8789) Use keras.layers.dense instead.
(pid=8789) WARNING:tensorflow:From /home/sj/.adt/venv-adt/lib/python3.6/site-packages/tensorflow/pyth

# All Rollout results are DIFFERENT as weights are all different!

In [9]:
for r_idx,rollout_res in enumerate(rollout_res_list):
    print ("Rollout result of [%d] worker is:\n %s"%(r_idx,rollout_res))

Rollout result of [0] worker is:
 [[-0.3  -0.36 -0.23 -0.92 -0.14  1.02  0.42  0.4 ]]
Rollout result of [1] worker is:
 [[-0.2  -0.52 -0.14  0.18 -0.28  0.16 -1.03 -0.04]]
Rollout result of [2] worker is:
 [[ 0.37 -0.9  -0.31  0.1  -0.63 -0.11 -0.09  0.77]]
Rollout result of [3] worker is:
 [[-0.59 -0.5  -0.32 -0.11 -0.52  1.13  0.23  0.21]]
Rollout result of [4] worker is:
 [[ 0.3   0.02  1.38 -0.16 -0.02  0.63 -0.5  -0.12]]


# Assign the same weights to all workers 

In [10]:
set_weights_list = [worker.set_weights.remote(weights) for worker in workers] # non-block
get_weights_list = [worker.get_weights.remote() for worker in workers] # non-block
weights_list = ray.get(get_weights_list)

### Rollouts and Check the results

In [11]:
rollout_list = [worker.rollout.remote(x=x_rand) for worker in workers] # non-block
rollout_res_list = ray.get(rollout_list)

In [12]:
for r_idx,rollout_res in enumerate(rollout_res_list):
    print ("Rollout result of [%d] worker is:\n %s"%(r_idx,rollout_res))

Rollout result of [0] worker is:
 [[-0.39 -0.58  1.48  0.05  0.88 -0.12  1.06  0.2 ]]
Rollout result of [1] worker is:
 [[-0.39 -0.58  1.48  0.05  0.88 -0.12  1.06  0.2 ]]
Rollout result of [2] worker is:
 [[-0.39 -0.58  1.48  0.05  0.88 -0.12  1.06  0.2 ]]
Rollout result of [3] worker is:
 [[-0.39 -0.58  1.48  0.05  0.88 -0.12  1.06  0.2 ]]
Rollout result of [4] worker is:
 [[-0.39 -0.58  1.48  0.05  0.88 -0.12  1.06  0.2 ]]


### All the Rollout resutls are the SAME!

### Shutdown RAY

In [13]:
ray.shutdown()
print ("RAY shutdown.")

RAY shutdown.
